In [1]:
import os
CODE_DIR = '/home/joshua/Projects/diffae'
os.chdir(f'{CODE_DIR}')

In [48]:
import torch
from torchvision.utils import save_image
from templates import *
from templates_cls import *
from experiment_classifier import ClsModel
import clip
from PIL import Image

import sys
sys.path.append('/home/joshua/Projects/diffae/disentanglement/')
from Losses import id_loss

device ="cuda" if torch.cuda.is_available() else "cpu"

#----------------Load FaRL and Arcface model----------------------#
FaRL_model, preprocess = clip.load("ViT-B/16", device="cpu")
FaRL_model = FaRL_model.to(device)
FaRL_state=torch.load("FaRL/checkpoints/FaRL-Base-Patch16-LAIONFace20M-ep64.pth") # you can download from https://github.com/FacePerceiver/FaRL#pre-trained-backbones
FaRL_model.load_state_dict(FaRL_state["state_dict"],strict=False)
FaRL_model.eval()

def extract_features(image_folder, save_folder):
    # 创建保存文件夹
    os.makedirs(save_folder, exist_ok=True)
    # 创建保存文件夹名称的txt文件
    txt_file = os.path.join(save_folder, "folder_names.txt")
    with open(txt_file, "w") as f:
        pass

    # 图像预处理
    FaRL_transform0 = transforms.Compose([
        transforms.Resize(size=224, interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(size=224),
        transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), 
                             std=(0.26862954, 0.26130258, 0.27577711))
    ])

    # 遍历图片文件夹
    for root, dirs, files in os.walk(image_folder):
        for file in files:
            # 读取图片
            image_path = os.path.join(root, file)
            image = Image.open(image_path).convert("RGB")
            # 图像预处理
            img_preprocess = transforms.ToTensor()
            image_tensor = img_preprocess(image).unsqueeze(0).cuda()

            # 使用模型提取特征
            with torch.no_grad():
                features = FaRL_model.visual(FaRL_transform0(image_tensor))

            # 保存图片和特征
            image_name = os.path.splitext(file)[0]
            # 创建保存图片和特征的子文件夹
            group_folder = os.path.join(save_folder, image_name)
            os.makedirs(group_folder, exist_ok=True)
            image_save_path = os.path.join(group_folder, image_name + ".jpg")
            feature_save_path = os.path.join(group_folder, image_name + ".pt")

            image.save(image_save_path)
            torch.save(features, feature_save_path)

            # 写入文件夹名称到txt文件
            with open(txt_file, "a") as f:
                f.write(group_folder + "\n")



In [49]:
# 示例用法
image_folder = "imgs_align"  # 输入图片文件夹
save_folder = "save_folder"  # 保存文件夹

extract_features(image_folder, save_folder)